In [5]:
!pip install scipy

In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/asheniranga/leaf-disease-dataset-combination')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  naklinaam


Your Kaggle Key:

  ································


100%|██████████| 761M/761M [00:33<00:00, 23.9MB/s] 


In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image

model = VGG19(weights='imagenet', include_top=False)

In [26]:
def extract_features_from_folders(folder_paths):
    image_paths = []
    features_list = []

    for folder_path in folder_paths:
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                    img_path = os.path.join(root, file)
                    img = image.load_img(img_path, target_size=(224, 224))  # Resize the image to VGG16's input size
                    img_array = image.img_to_array(img)
                    img_array = np.expand_dims(img_array, axis=0)
                    img_array = preprocess_input(img_array)

                    features = model.predict(img_array)
                    features_list.append(features)
                    image_paths.append(img_path)

    return features_list, image_paths

In [27]:
top_level_folders = ['/kaggle/working/leaf-disease-dataset-combination/image data/train/peach','/kaggle/working/leaf-disease-dataset-combination/image data/train/Rice','/kaggle/working/leaf-disease-dataset-combination/image data/train/cherry (including sour)','/kaggle/working/leaf-disease-dataset-combination/image data/train/strawberry','/kaggle/working/leaf-disease-dataset-combination/image data/train/tomato','/kaggle/working/leaf-disease-dataset-combination/image data/train/orange','/kaggle/working/leaf-disease-dataset-combination/image data/train/squash','/kaggle/working/leaf-disease-dataset-combination/image data/train/Cassava','/kaggle/working/leaf-disease-dataset-combination/image data/train/corn (maize)','/kaggle/working/leaf-disease-dataset-combination/image data/train/grape','/kaggle/working/leaf-disease-dataset-combination/image data/train/potato','/kaggle/working/leaf-disease-dataset-combination/image data/train/apple','/kaggle/working/leaf-disease-dataset-combination/image data/train/pepper, bell']  


In [28]:
image_features, image_paths = extract_features_from_folders(top_level_folders)


1/1 [==============================] - 0s 17ms/step


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1/1 [==============================] - 0s 18ms/step


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1/1 [==============================] - 0s 19ms/step


In [70]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Dropout

image_features = np.random.random((100, 10, 4096))  # (num_samples, sequence_length, feature_dim)
labels = np.random.randint(2, size=(100,))

split_ratio = 0.8
split_index = int(len(image_features) * split_ratio)
x_train, x_val = image_features[:split_index], image_features[split_index:]
y_train, y_val = labels[:split_index], labels[split_index:]




In [77]:
model = keras.Sequential()
model.add(LSTM(64, input_shape=(x_train.shape[1], x_train.shape[2], )))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [78]:
model.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_val, y_val))


Epoch 1/30
3/3 [==============================] - 2s 237ms/step - loss: 1.5792 - accuracy: 0.4750 - val_loss: 0.5204 - val_accuracy: 0.8000
Epoch 2/30
3/3 [==============================] - 0s 22ms/step - loss: 0.7528 - accuracy: 0.4750 - val_loss: 0.7181 - val_accuracy: 0.2000
Epoch 3/30
3/3 [==============================] - 0s 20ms/step - loss: 0.6966 - accuracy: 0.4750 - val_loss: 0.7271 - val_accuracy: 0.2000
Epoch 4/30
3/3 [==============================] - 0s 19ms/step - loss: 0.6971 - accuracy: 0.4750 - val_loss: 0.7277 - val_accuracy: 0.2000
Epoch 5/30
3/3 [==============================] - 0s 20ms/step - loss: 0.6972 - accuracy: 0.4750 - val_loss: 0.7272 - val_accuracy: 0.2000
Epoch 6/30
3/3 [==============================] - 0s 19ms/step - loss: 0.6970 - accuracy: 0.4750 - val_loss: 0.7259 - val_accuracy: 0.2000
Epoch 7/30
3/3 [==============================] - 0s 20ms/step - loss: 0.6965 - accuracy: 0.4750 - val_loss: 0.7224 - val_accuracy: 0.2000
Epoch 8/30
3/3 [==========

In [79]:

# Evaluate the model
loss, accuracy = model.evaluate(x_val, y_val)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

1/1 [==============================] - 0s 26ms/step - loss: 0.6904 - accuracy: 0.7000
Validation Loss: 0.690416157245636, Validation Accuracy: 0.699999988079071


In [80]:
from tensorflow.keras.models import load_model

model.save('/kaggle/working/lstmmodel.h5')